<a href="https://colab.research.google.com/github/piusijabla/CH4_isotope_precision_analysis/blob/main/CH4_isotope_precision_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MScR_Code
Import package
%matplotlib inline
from scipy.interpolate import interp1d

import matplotlib as mpl
import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
mpl.rcParams['axes.formatter.useoffset'] = False
hmax = lambda ax: ax.set_major_formatter(md.DateFormatter('(%d) %H:%M'))
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#plt.style.use(['seaborn-whitegrid', 'presentation'])
mpl.rcParams['figure.dpi'] = 300


del13_label = '$\delta ^{13}\mathrm{C}\, (\mathrm{CH}_4)$ [‰]'
delD_label = '$\delta ^{2}\mathrm{H} (\mathrm{CH}_4)$ [‰]'

def top_legend(ax, *args, **kwargs):
    n_lines = len(ax.get_lines())
    ax.legend(*args, loc='lower right', bbox_to_anchor=(1., 1), ncol=n_lines, **kwargs)

def outer_legend(ax, *args, **kwargs):
    ax.legend(*args, loc='upper right', bbox_to_anchor=(1, 0.5), **kwargs)

def save_fig(fig, name):
    fig.savefig('figures/'+name+'.png', dpi=300)
Import the raw data from GCWerks
df = pd.read_csv(r'T:\EAM\TECHNICAL\Heathfield\Technical\Data\GHG_Data\Delta\fulldata\gcwerks_raw.csv', skipinitialspace=True)

keep_columns = ['time', 'type', 'port', 'sample',
                '13ch4_meas', '13ch4_sens', '13ch4_Ctank',
                '12ch4_meas', '12ch4_sens', '12ch4_Ctank',
                '12ch4_a_meas', '12ch4_a_sens', '12ch4_a_Ctank',
                'ch3d_meas', 'ch3d_sens', 'ch3d_Ctank','cavity_press','cavity_temp','chisq_1','chisq_2','TLaser1','TLaser2','VLaser1','VLaser2','Range_F1_L1','Range_F1_L2','pos1']
df = pd.read_csv(r'T:\EAM\TECHNICAL\Heathfield\Technical\Data\GHG_Data\Delta\fulldata\gcwerks_raw.csv', skipinitialspace=True, parse_dates=['time'], index_col=['time', 'type'], usecols=keep_columns)
df['type'] = df.index.get_level_values(1)
df.head()

Drop Duplicate
#df[df.index.duplicated()].index
duplicated_rows = [
            '2022-06-14 18:14:00',
            '2022-06-15 02:14:00',
            '2022-06-15 10:14:00',
            '2022-06-18 19:31:00',
            '2022-06-19 05:31:00',
            '2022-06-19 13:31:00',
            '2022-06-19 21:32:00']
df.drop(duplicated_rows, inplace=True, errors='ignore')
We need the isotopologue amount fraction for the Boreas Target and calibration tanks. Fix this by using the sensitivity that is calculated from the measurement and tank value, and this is exported by GCWerks:
$$ sens = \frac{meas}{Ctank} $$
Remove this step when exporting tank values is fixed.
df['12ch4_Ctank'] = df['12ch4_meas']/df['12ch4_sens']
df['13ch4_Ctank'] = df['13ch4_meas']/df['13ch4_sens']
df['ch3d_Ctank'] = df['ch3d_meas']/df['ch3d_sens']

df.xs('low_std', level=1)[['12ch4_Ctank', '13ch4_Ctank', 'ch3d_Ctank']].dropna().head()

Separate Aerodyne and Preconcentrator data

We now need to gather the 3 rows that make up each preconcentrator cycle so that the data processing steps can be applied row-by-row. The measured values are moved into new columns.

Make a new data frame containing only the Aerodyne samples, using a list of the names for these in the `type` column. Then extract these rows into columns and name the columns with `"type"<space>"measurement`.

aerodyne_type = ['low_std', 'med_std', 'hi_tank']
aerodyne_data = df.unstack().swaplevel(axis=1)[aerodyne_type]
aerodyne_data.drop('type', axis=1, level=1, inplace=True)
aerodyne_data.columns = aerodyne_data.columns.map(' '.join)
aerodyne_data.head()

Create a second data frame containing only the preconcentrated sample data - do this by dropping all rows that match an aerodyne run type so that anything remaining is a run that we need to calibrate. Then clean up the column names by first stacking and then dropping the extra column index.

precon_data = df.unstack().swaplevel(axis=1).drop(aerodyne_type, level=0, axis=1).stack(level=0).droplevel(level=1, axis=0)
precon_data.head()

Merge data row by row

This step builds the complete data frame. The final data needed is the trapping volume recorded for each sample, which is saved in a separated CSV file with timestamps that match the rest of the data. Load this first.

vtrap_df = pd.read_csv(r'T:\EAM\TECHNICAL\Heathfield\Technical\Data\GHG_Data\Delta\fulldata\vtrap.csv', parse_dates=['time'], index_col='time')
vtrap_df['v_trap']
Merge the Aerodyne and preconcentrated sample dataframes into a single one. This makes a new dataframe where each row corresponds to a single Boreas trapping cycle; the named columns correspond to the preconcentrated sample, e.g. 12ch4_meas; and the Aerodyne calibration samples have the Aerodyne run type prepended to the column name, e.g. low_tank 12ch4_meas. Specifying the indicies ensures that only rows where the timestamp matches in both data frames are merged.

boreas_data = precon_data.merge(vtrap_df, how='left', left_index=True, right_index=True)
boreas_data = boreas_data.merge(aerodyne_data, left_index=True, right_index=True)

Clean the data by dropping rows that are flagged as NaN, rows where v_trap has not been correctly logged and remove unneeded columns. Print out the first few runs to check.

unused_columns = ['12ch4_sens', '13ch4_sens', 'ch3d_sens', '12ch4_a_sens']

boreas_data.dropna(subset=['12ch4_meas', '13ch4_meas', 'ch3d_meas'], inplace=True, how='all')
boreas_data.drop(unused_columns, axis=1, inplace=True)
boreas_data.drop(boreas_data[boreas_data['v_trap'] > 6000].index, inplace=True)

boreas_data.head()


Calibrate amount fraction in the spectrometer

This step calibrates the amount fraction of the preconcentrated sample and the high concentration mixtures (`hi_tank`) in the spectrometer. First calculate the interpolation weighting from the Aerodyne response:
$$ \alpha = \frac{R^\mathrm{samp} - R^\mathrm{low}}{R^\mathrm{high} - R^\mathrm{low}} $$
Then, the calibrated amount fraction is the average of the tank values weighted by this interpolation:

$$ Y^\mathrm{samp} = (1-\alpha)Y^\mathrm{low} + \alpha Y^\mathrm{high} $$

Define a function that perfoms this calculation for a given isotopologue and sample name by concatenating the names to look up the correct colunms, then write the output to a new column.

def interpolate(df, iso_name, samp_name):
    # Build the name string for the sample
    samp_meas = samp_name + iso_name + '_meas'
    samp_cal = samp_name + iso_name + '_cal'
    samp_alpha = samp_name + iso_name + '_alpha'

    # Build the name string for the standards
    low_std_meas = 'low_std ' + iso_name + '_meas'
    med_std_meas = 'med_std ' + iso_name + '_meas'
    low_std_ctank = 'low_std ' + iso_name + '_Ctank'
    med_std_ctank = 'med_std ' + iso_name + '_Ctank'

    # Calculate the interpolation
    df[samp_alpha] = (df[samp_meas] - df[low_std_meas])/(df[med_std_meas] - df[low_std_meas])
    df[samp_cal] = (1 - df[samp_alpha]) * df[low_std_ctank] + df[samp_alpha] * df[med_std_ctank]

Loop over isotopologue names and sample names and calculate the interpolation. The blank sample name `''` is the preconcentrated sample (no extra words on the data e.g. `12ch4_meas`), and there is a space after the Aerodyne names `'hi_tank '` to make the function work by matching the column names that are searched for in the data frame (these are named, .e.g., `hi_tank 12ch4_meas`).

for iso_name in ['12ch4', '13ch4', 'ch3d']:
    for samp_name in ['', 'hi_tank ']:
        interpolate(boreas_data, iso_name, samp_name)
boreas_data.head()

Calculate Delta from isotope ratio

boreas_data['r13_cal'] = boreas_data['13ch4_cal']/boreas_data['12ch4_cal']
boreas_data['r2_cal'] = boreas_data['ch3d_cal']/boreas_data['12ch4_cal']
boreas_data['r13_tank'] = boreas_data['13ch4_Ctank']/boreas_data['12ch4_Ctank']
boreas_data['r2_tank'] = boreas_data['ch3d_Ctank']/boreas_data['12ch4_Ctank']

boreas_data['hi_tank r13_cal'] = boreas_data['hi_tank 13ch4_cal']/boreas_data['hi_tank 12ch4_cal']
boreas_data['hi_tank r2_cal'] = boreas_data['hi_tank ch3d_cal']/boreas_data['hi_tank 12ch4_cal']

The delta values are calculated from the isotopologue ratios and stored as per-mille.

r13_vpdb   = 0.0111802
r2_vsmow   = 0.00015575
boreas_data['del13 VPDB'] = (boreas_data['r13_cal'] / r13_vpdb - 1) * 1000
boreas_data['delD VSMOW'] = (0.25 * boreas_data['r2_cal'] / r2_vsmow  - 1) * 1000

boreas_data['hi_tank del13 VPDB'] = (boreas_data['hi_tank r13_cal'] / r13_vpdb - 1)* 1000
boreas_data['hi_tank delD VSMOW'] = (0.25 * boreas_data['hi_tank r2_cal'] / r2_vsmow  - 1) * 1000


Plot

fig, ax = plt.subplots(nrows=2, sharex=True)
air_data = boreas_data[boreas_data['type']=='air']
bt_data = boreas_data[boreas_data['type']=='BT']
ax[0].plot(air_data['del13 VPDB'], '.', ms=3, label='air')
ax[1].plot(air_data['delD VSMOW'], '.', ms=3)
ax[0].plot(bt_data['del13 VPDB'], '.', ms=3, label='BT')
ax[1].plot(bt_data['delD VSMOW'], '.', ms=3)
ax[1].set_xlabel(' Year - Month - Day', fontsize='10')

top_legend(ax[0])
ax[0].set_ylabel(del13_label)
ax[1].set_ylabel(delD_label)
ax[1].grid()
ax[0].grid()
ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
fig.autofmt_xdate()



fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(boreas_data['hi_tank del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[1].plot(boreas_data['hi_tank delD VSMOW'], '.', ms=1)



top_legend(ax[0])
ax[0].set_ylabel(del13_label)
ax[1].set_ylabel(delD_label)
ax[0].set_ylim(-57,-40)
ax[1].set_ylim(-200,-180)
ax[1].set_xlabel(' Year - Month - Day', fontsize='10')
ax[1].grid()
ax[0].grid()
ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
fig.autofmt_xdate()

When Laser was at NPL Temperature controlled Lab

boreas_data_NPL = pd.read_csv(r'C:\Users\ipp\OneDrive - National Physical Laboratory\Desktop\PhD Research\PhD_data\boreas_data_NPL.csv', skipinitialspace=True, parse_dates=['time'], index_col=['time'])
boreas_data_NPL

fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(boreas_data_NPL['hi_tank del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[1].plot(boreas_data_NPL['hi_tank delD VSMOW'], '.', ms=1)


top_legend(ax[0])
ax[0].set_ylabel(del13_label)
ax[1].set_ylabel(delD_label)
ax[0].set_ylim(-43,-40)
ax[1].set_ylim(-195,-186)
ax[1].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()
ax[1].grid()
ax[-1].set_xlim(pd.to_datetime('2021-01-08'), pd.to_datetime('2021-02-15'))
fig.autofmt_xdate()

fig, ax = plt.subplots(nrows=3, sharex=True)

ax[0].plot(boreas_data_NPL['hi_tank 12ch4_meas']/1000, '.r', ms=1, label= '$^{12}\mathrm{CH_4}$')
ax[1].plot(boreas_data_NPL['hi_tank ch3d_meas']/1000, '.g', ms=1, label = '$^{12}\mathrm{CH_3D}$')
ax[2].plot(boreas_data_NPL['hi_tank 13ch4_meas']/1000, '.', ms=1, label = '$^{13}\mathrm{CH_4}$')



ax[1].set_ylabel('Amount Fraction of $\mathrm{CH}_4$ Isotoplogue (ppm)')
ax[0].set_ylim(600,620)
ax[1].set_ylim(500,550)
ax[2].set_ylim(550,600)
ax[2].set_xlabel('Year - Month - Day', fontsize='10')
ax[0].grid()

ax[1].grid()

ax[2].grid()
outer_legend(ax[0])
outer_legend(ax[1])
outer_legend(ax[2])
ax[-1].set_xlim(pd.to_datetime('2021-01-08'), pd.to_datetime('2021-02-15'))
fig.autofmt_xdate()


Precision when at NPL
#precision = '4'
boreas_data_NPL['std_del13 VPDB']=boreas_data_NPL['hi_tank del13 VPDB'].rolling(4).std()
boreas_data_NPL['std_delD VSMOW']=boreas_data_NPL['hi_tank delD VSMOW'].rolling(4).std()

fig, ax = plt.subplots(nrows=3, sharex=True)

ax[0].plot(boreas_data_NPL['std_del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[2].plot(boreas_data_NPL['hi_tank TLaser1'], '.', ms=1)
ax[1].plot(boreas_data_NPL['hi_tank pos1'], '.g', ms=1)

top_legend(ax[0])
ax[0].set_ylabel(del13_label, fontsize='8')
ax[2].set_ylabel('Laser Temperature (℃)', fontsize='8')
ax[1].set_ylabel('Laser Position', fontsize='8')
ax[0].set_ylim(-0.03,0.5)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[-1].set_xlim(pd.to_datetime('2021-01-08'), pd.to_datetime('2021-02-15'))
fig.autofmt_xdate()

fig, ax = plt.subplots(nrows=3,sharex=True)

ax[0].plot(boreas_data_NPL['std_del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[1].plot(boreas_data_NPL['hi_tank cavity_press'], '.', ms=1)
ax[2].plot(boreas_data_NPL['hi_tank cavity_temp'], '.g', ms=1)

top_legend(ax[0])
ax[0].set_ylabel(del13_label, fontsize='8')
ax[1].set_ylabel('Pressure (torr)', fontsize='8')
ax[2].set_ylabel('Temperature (K)', fontsize='8')
ax[0].set_ylim(-0.01,0.5)
ax[1].set_ylim(21,22.5)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
#ax[2].set_ylim(290.2,290.6)
ax[0].grid()

ax[1].grid()

ax[2].grid()
ax[-1].set_xlim(pd.to_datetime('2021-01-08'), pd.to_datetime('2021-02-15'))
fig.autofmt_xdate()


fig, ax = plt.subplots(nrows=3,sharex=True)

ax[0].plot(boreas_data_NPL['std_del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[1].plot(boreas_data_NPL['hi_tank chisq_1'], '.', ms=1)
ax[2].plot(boreas_data_NPL['hi_tank pos1'], '.g', ms=1)

top_legend(ax[0])
ax[0].set_ylabel(del13_label, fontsize='8')
ax[1].set_ylabel('chisq', fontsize= '8')
ax[2].set_ylabel('Laser Position', fontsize= '8')
ax[0].set_ylim(-0.01,0.5)
#ax[1].set_ylim(200,400)
#ax[2].set_ylim(231.5,238)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()

ax[1].grid()

ax[2].grid()
#ax[-1].set_xlim(pd.to_datetime('2021-01-08'), pd.to_datetime('2021-02-15'))
fig.autofmt_xdate()

Precision
#precision = '4'
boreas_data['std_del13 VPDB']=boreas_data['hi_tank del13 VPDB'].rolling(4).std()
boreas_data['std_delD VSMOW']=boreas_data['hi_tank delD VSMOW'].rolling(4).std()

fig, ax = plt.subplots(nrows=3, sharex=True)

ax[0].plot(boreas_data['std_del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[2].plot(boreas_data['hi_tank TLaser1'], '.', ms=1)
ax[1].plot(boreas_data['hi_tank pos1'], '.g', ms=1)

top_legend(ax[0])
ax[0].set_ylabel(del13_label, fontsize='8')
ax[2].set_ylabel('Laser Temperature (℃)', fontsize='8')
ax[1].set_ylabel('Laser Position', fontsize= '8')
ax[0].set_ylim(-0.03,0.55)
ax[2].set_ylim(-15.650,-15.5)
ax[1].set_ylim(231.5,238)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()

ax[1].grid()

ax[2].grid()

ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
fig.autofmt_xdate()

fig, ax = plt.subplots(nrows=3,sharex=True)

ax[0].plot(boreas_data['std_del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[1].plot(boreas_data['hi_tank cavity_press'], '.', ms=1)
ax[2].plot(boreas_data['hi_tank cavity_temp'], '.g', ms=1)

top_legend(ax[0])
ax[0].set_ylabel(del13_label, fontsize='8')
ax[1].set_ylabel('Pressure (torr)', fontsize= '8')
ax[2].set_ylabel('Temperature (K)', fontsize= '8')
ax[0].set_ylim(-0.01,0.5)
ax[1].set_ylim(17.9,18.25)
ax[2].set_ylim(290.2,290.6)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()

ax[1].grid()

ax[2].grid()
ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
fig.autofmt_xdate()

fig, ax = plt.subplots(nrows=3,sharex=True)

ax[0].plot(boreas_data['std_del13 VPDB'], '.r', ms=1, label='hi_tank')
ax[1].plot(boreas_data['hi_tank chisq_1'], '.', ms=1)
ax[2].plot(boreas_data['hi_tank pos1'], '.g', ms=1)

top_legend(ax[0])
ax[0].set_ylabel(del13_label, fontsize='8')
ax[1].set_ylabel('chisq', fontsize= '8')
ax[2].set_ylabel('Laser Position', fontsize= '8')
ax[0].set_ylim(-0.01,0.5)
ax[1].set_ylim(200,400)
ax[2].set_ylim(231.5,238)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()

ax[1].grid()

ax[2].grid()
ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
fig.autofmt_xdate()

fig, ax = plt.subplots(nrows=3, sharex=True)

ax[0].plot(boreas_data['hi_tank 12ch4_meas']/1000, '.r', ms=1, label= '$^{12}\mathrm{CH_4}$')
ax[2].plot(boreas_data['hi_tank 13ch4_meas']/1000, '.', ms=1, label = '$^{13}\mathrm{CH_4}$')
ax[1].plot(boreas_data['hi_tank ch3d_meas']/1000, '.g', ms=1, label = '$^{12}\mathrm{CH_3D}$')



ax[1].set_ylabel('Amount Fraction of $\mathrm{CH}_4$ Isotoplogue (ppm)')
ax[0].set_ylim(510,560)
ax[2].set_ylim(520,540)
ax[1].set_ylim(450,500)
ax[2].set_xlabel(' Year - Month - Day', fontsize='10')
ax[0].grid()

ax[1].grid()

ax[2].grid()

ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
outer_legend(ax[0])
outer_legend(ax[1])
outer_legend(ax[2])

fig.autofmt_xdate()


Stability

stability_duration = '24h'

bt_data = boreas_data[boreas_data['type']=='BT']
bt_stability = bt_data[['del13 VPDB', 'delD VSMOW']].rolling(stability_duration).std()

stability = boreas_data[['hi_tank del13 VPDB', 'hi_tank delD VSMOW']].rolling(stability_duration).std()
fig, ax = plt.subplots(nrows=2, sharex=True)
ax[0].plot(stability['hi_tank del13 VPDB'], '.', ms=1, label='hi_tank')
ax[1].plot(stability['hi_tank delD VSMOW'], '.', ms=1)

ax[0].set_ylabel(del13_label)
ax[1].set_ylabel(delD_label)
ax[0].set_ylim(-0.01, 0.2)
ax[1].set_ylim(-0.1, 2)
ax[1].set_xlabel(' Year - Month - Day', fontsize='10')
ax[-1].set_xlim(pd.to_datetime('2023-04-01'), pd.to_datetime('2023-08-10'))
ax[0].grid()

ax[1].grid()

top_legend(ax[0])
fig.autofmt_xdate()
fig.tight_layout()


Allan Variance

raw_data = pd.read_csv(r"T:\EAM\TECHNICAL\Heathfield\Technical\Data\Aerodyne Stability\200802_str.csv")
raw_data.head()


Get the mean, variance and mean_variance of the 12ch4 concentration column

A = np.mean(raw_data['12CH4']) #mean
print(A)
var_A = np.var(raw_data['12CH4']) #variance
print(var_A)
N = len(raw_data['12CH4'])
A_n = var_A/N #mean_variance
print(A_n)


Divide N into M subgroup containing k element¶

k=100
M = N/k

print(M)


calcualte the mean of the subgroup M

# Function to calculate the mean of a subgroup 'M' containing 'k' elements
def subgroup_mean(data, k):
    means = []  # List to store the means of subgroups

    # Loop through the DataFrame in steps of 'k' elements
    for i in range(0, len(data), k):
        subgroup = data.iloc[i:i + k]  # Extract the subgroup
        subgroup_mean = subgroup.mean()  # Calculate the mean of the subgroup
        means.append(subgroup_mean)  # Append the mean to the 'means' list

    return means

# Call the function to get the means of subgroups with size 'k'
k = 100  # Change 'k' to your desired subgroup size
means_of_subgroups = subgroup_mean(raw_data['12CH4'], k)

#print(means_of_subgroups)



Get allan variance

## For 12CH4
k_12ch4 = np.arange(1, 200, 1) # define subgroup range
a_variance_12 = []
for kappa in k_12ch4:
    k_mean = subgroup_mean(raw_data['12CH4'], kappa) # calculate mean for subgroup
    #print(k_mean)

    m = N/kappa # recalculate M
    # calculate Alan variance for subgroup
    sigma = np.sum((np.array(k_mean[1:-1])-np.array(k_mean[0:-2]))**2)
    sigma = sigma/2/(m-1)
    a_variance_12.append(np.sqrt(sigma))
    #print(a_variance)
    #print((np.array(k_mean[1:-1])-np.array(k_mean[0:-2]))**2)


### For 13CH4
k_13ch4 = np.arange(1, 200, 1) # define subgroup range
a_variance_13 = []
for kappa in k_13ch4:
    k_mean = subgroup_mean(raw_data['13CH4'], kappa) # calculate mean for subgroup
    #print(k_mean)

    m = N/kappa # recalculate M
    # calculate Alan variance for subgroup
    sigma = np.sum((np.array(k_mean[1:-1])-np.array(k_mean[0:-2]))**2)
    sigma = sigma/2/(m-1)
    a_variance_13.append(np.sqrt(sigma))


### For CH3D
k_ch3d = np.arange(1, 200, 1) # define subgroup range
a_variance_ch3d = []
for kappa in k_ch3d:
    k_mean = subgroup_mean(raw_data['CH3D'], kappa) # calculate mean for subgroup
    #print(k_mean)

    m = N/kappa # recalculate M
    # calculate Alan variance for subgroup
    sigma = np.sum((np.array(k_mean[1:-1])-np.array(k_mean[0:-2]))**2)
    sigma = sigma/2/(m-1)
    a_variance_ch3d.append(np.sqrt(sigma))

## Plots
fig, ax = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(15,5))
plt.subplot(1, 3, 1)
ax[0].loglog(k_12ch4, a_variance_12, color='orange')
ax[0].grid()
ax[0].set_ylabel('Allan Deviation')
ax[0].grid(which='major', linewidth=0.8)
ax[0].grid(which='minor', linestyle=':', linewidth=0.5)
ax[0].minorticks_on()

plt.subplot(1, 3, 2)
ax[1].loglog(k_13ch4, a_variance_13)
ax[1].set_xlabel('Integration Time (s)')
ax[1].grid()
ax[1].grid(which='major', linewidth=0.8)
ax[1].grid(which='minor', linestyle=':', linewidth=0.5)
ax[1].minorticks_on()
plt.subplot(1, 3, 3)
ax[2].loglog(k_ch3d, a_variance_ch3d,  color='green')
ax[2].grid()
ax[2].grid(which='major', linewidth=0.8)
ax[2].grid(which='minor', linestyle=':', linewidth=0.5)
ax[2].minorticks_on()
ax[0].set_title('$^{12}\mathrm{CH_4}$')
ax[1].set_title('$^{13}\mathrm{CH_4}$')
ax[2].set_title('$^{12}\mathrm{CH_3D}$')
plt.show()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Compute the ratios and add to the DataFrame
raw_data['13R'] = raw_data['13CH4'] / raw_data['12CH4']
raw_data['2R'] = raw_data['CH3D'] / raw_data['12CH4']

def compute_allan_variance(data, alpha):
    def subgroup_mean(data, alpha):
        return [np.mean(data[i:i+alpha]) for i in range(0, len(data), alpha)]

    N = len(data)
    k_mean = subgroup_mean(data, alpha)
    m = N // alpha
    sigma = np.sum((np.array(k_mean[1:]) - np.array(k_mean[:-1]))**2)
    sigma = sigma / 2 / (m - 1)
    return np.sqrt(sigma)

# Set the range of alphas from 1 to 500
alphas = np.arange(1, 1500, 1)

optimum_deviations_13R = []
optimum_alphas_13R = []

optimum_deviations_2R = []
optimum_alphas_2R = []

# Loop over alphas and compute Allan variance
for alpha in alphas:
    deviation_13R = compute_allan_variance(raw_data['13R'], alpha)
    deviation_2R = compute_allan_variance(raw_data['2R'], alpha)

    optimum_deviations_13R.append(deviation_13R)
    optimum_alphas_13R.append(alpha)

    optimum_deviations_2R.append(deviation_2R)
    optimum_alphas_2R.append(alpha)

# Create a single plot for the range of alphas from 1 to 500
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(12, 6))

ax[0].loglog(optimum_alphas_13R, optimum_deviations_13R, color='orange')
ax[1].loglog(optimum_alphas_2R, optimum_deviations_2R, color='green')

# Add annotations, labels, and other plot settings here
ax[0].set_ylabel('Allan Deviation')
ax[0].set_xlabel('Integration Time (s)')
ax[1].set_xlabel('Integration Time (s)')
ax[0].set_title('$^{13}\mathrm{R} $')
ax[1].set_title('$^{2}\mathrm{R} $')

for axis in ax:
    axis.grid(which='major', linewidth=0.8)
    axis.grid(which='minor', linestyle=':', linewidth=0.5)
    axis.minorticks_on()

plt.tight_layout()
plt.savefig('alpha_1_to_1500_plots.png')
plt.show()
plt.close()

import pandas as pd
import matplotlib.pyplot as plt

# Constants for calculations
r13_ref = 0.01118
r2_ref = 0.00015575
r13_hitran = 0.0112374
r2_hitran = 0.000156

# Reading the data from the uploaded file
raw_data = pd.read_csv(r"T:\EAM\TECHNICAL\Heathfield\Technical\Data\Aerodyne Stability\200802_str.csv", skipinitialspace=True, parse_dates=['date'], index_col=['date'])

# Method 1: 500-second averaging
data_500s = raw_data.resample('500S').mean()
data_500s['13r'] = data_500s['13CH4'] / data_500s['12CH4']
data_500s['2r'] = data_500s['CH3D'] / data_500s['12CH4']
data_500s['Rsum'] = (1 + data_500s['13r']) * ((1 + data_500s['2r'])**4)
data_500s['X211'] = 1 / data_500s['Rsum']
data_500s['X311'] = data_500s['13r'] / data_500s['Rsum']
data_500s['X212'] = (4 * data_500s['2r']) / data_500s['Rsum']
data_500s['delta_13C'] = ((data_500s['X311'] / data_500s['X211']) * r13_hitran / r13_ref) - 1
data_500s['delta_2H'] = (0.25*(data_500s['X212'] / data_500s['X211']) * r2_hitran / r2_ref) - 1
data_500s['delta_13C'] *= 1000
data_500s['delta_2H'] *= 1000
data_500s['delta_13C_std'] = data_500s['delta_13C'].rolling(window=4).std()
data_500s['delta_2H_std'] = data_500s['delta_2H'].rolling(window=4).std()
precision_500s_delta_13C = data_500s['delta_13C_std'].mean()
precision_500s_delta_2H = data_500s['delta_2H_std'].mean()

# Method 2: 100-second averaging for individual isotopologues
data_100s = raw_data.resample('100S').mean()
data_100s['13r'] = data_100s['13CH4'] / data_100s['12CH4']
data_100s['2r'] = data_100s['CH3D'] / data_100s['12CH4']
data_100s['Rsum'] = (1 + data_100s['13r']) * ((1 + data_100s['2r'])**4)
data_100s['X211'] = 1 / data_100s['Rsum']
data_100s['X311'] = data_100s['13r'] / data_100s['Rsum']
data_100s['X212'] = (4 * data_100s['2r']) / data_100s['Rsum']
data_100s['delta_13C'] = ((data_100s['X311'] / data_100s['X211']) * r13_hitran / r13_ref) - 1
data_100s['delta_2H'] = (0.25*(data_100s['X212'] / data_100s['X211']) * r2_hitran / r2_ref) - 1
data_100s['delta_13C'] *= 1000
data_100s['delta_2H'] *= 1000
data_100s['delta_13C_std'] = data_100s['delta_13C'].rolling(window=4).std()
data_100s['delta_2H_std'] = data_100s['delta_2H'].rolling(window=4).std()
precision_100s_delta_13C = data_100s['delta_13C_std'].mean()
precision_100s_delta_2H = data_100s['delta_2H_std'].mean()

# Reindexing the 100s data to match the 500s data
data_100s_reindexed = data_100s.reindex(data_500s.index)

# Plotting the 4-point rolling standard deviation (precision) for both averaging methods

# Setting up the figure for precision plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot a: 4-point rolling standard deviation of delta_13C for 500s averaging with mean precision
axes[0, 0].plot(data_500s.index, data_500s['delta_13C_std'],'.', label='$\delta ^{{13}}\mathrm{{C}} $', color='orange')
axes[0, 0].set_xlabel('Time (Month-Day-Hour)')
axes[0, 0].set_ylabel('Precision [‰]')
axes[0, 0].set_title('(a) 500s Averaging Time $\delta ^{{13}}\mathrm{{C}} $\nMean Precision: {:.4f}‰'.format(precision_500s_delta_13C))
axes[0, 0].grid(True)
axes[0, 0].legend()

# Plot b: 4-point rolling standard deviation of delta_2H for 500s averaging with mean precision
axes[0, 1].plot(data_500s.index, data_500s['delta_2H_std'],'.', label='$\delta ^{{2}}\mathrm{{H}} $', color='green')
axes[0, 1].set_xlabel('Time (Month-Day-Hour)')
axes[0, 1].set_ylabel('Precision [‰]')
axes[0, 1].set_title('(b) 500s Averaging Time $\delta ^{{2}}\mathrm{{H}} $\nMean Precision: {:.4f}‰'.format(precision_500s_delta_2H))
axes[0, 1].grid(True)
axes[0, 1].legend()

# Plot c: 4-point rolling standard deviation of delta_13C for 100s averaging with mean precision
axes[1, 0].plot(data_100s_reindexed.index, data_100s_reindexed['delta_13C_std'],'.', label='$\delta ^{{13}}\mathrm{{C}} $', color='blue')
axes[1, 0].set_xlabel('Time (Month-Day-Hour)')
axes[1, 0].set_ylabel('Precision [‰]')
axes[1, 0].set_title('(c) 100s Averaging Time $\delta ^{{13}}\mathrm{{C}} $\nMean Precision: {:.4f}‰'.format(precision_100s_delta_13C))
axes[1, 0].grid(True)
axes[1, 0].legend()

# Plot d: 4-point rolling standard deviation of delta_2H for 100s averaging with mean precision
axes[1, 1].plot(data_100s_reindexed.index, data_100s_reindexed['delta_2H_std'],'.', label='$\delta ^{{2}}\mathrm{{H}} $', color='purple')
axes[1, 1].set_xlabel('Time (Month-Day-Hour)')
axes[1, 1].set_ylabel('Precision [‰]')
axes[1, 1].set_title('(d) 100s Averaging Time $\delta ^{{2}}\mathrm{{H}} $\nMean Precision: {:.4f}‰'.format(precision_100s_delta_2H))
axes[1, 1].grid(True)
axes[1, 1].legend()
# Rotate x-axis labels by 45 degrees
for ax in axes.flat:
    ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()